---
format:
  html:
    css: styles/custom.css
editor: visual
author: Lorenzo Vives
toc: true
theme: cosmo
layout: full
---

## NBA project

Here you will find a little project about data analysis. Particularly, this project contains data manipulation, data visualization, OLS, Logit and MEM (Mixed Effects Models). I´m using an `nba` database which contains 5313 players from 1947-2025. You can find more details about this dataset [here](https://www.kaggle.com/datasets/flynn28/v2-nba-player-database/data).

I will be working with this variables:

`Name`: Players name

`Position`: Players position(s)

`Height`: Height of player (inches)

`Weight`: Weight of player (lbs)

`School`: School(s) player attended

`Active`: If player is currently playing (True or False)

`G`: amount of games played by player

`PTS`: average points scored by player per game

`TRB`: average rebounds by player per game

`AST`: average assists per game

## Research exercise

For this project, I will be analyzing the relationship between being a Center (here I will include those who can play two positions, for example Forward-Center or Center-Forward ) and scoring performance. Also, some predictors will be included, such as Games played, Height, Weight, average total asists per game (AST), average total rebounds per game (TRB).

## Data and libraries

So, let's get into the data! We will be working with the following libraries. Then we load our NBA data.

```{r}
library(dplyr)
library(ggplot2)
library(scales)
library(lme4)
library(misty)
library(texreg)
library(ggrepel)
library(tidyr)
library(lattice)
library(gridExtra)
library(ggthemes)

nba<- read.csv("D:/RSTUDIOWD/papers/nba-proyect/NBA_PLAYERS.csv")
```

## Variable processing in R

First, we clear our NA cases

```{r}
nba<- nba %>% 
  na.omit()
```

Now, let's take a look at the variables in the dataset

```{r}
colnames(nba)
```

For the Schools, we display the 10 schools with most cases

```{r}
colnames(nba)
nba %>% 
count(School, sort = T) %>% 
  head(10)
```

Here we observe that the variable with most cases is not labeled, but is not a missed value either. This means that these players did not come from any School. We label this situation as \`street\`.

```{r}
nba <- nba %>%
  mutate(School = ifelse(School == "" | is.na(School), "street", School))
```

For \`Height\` and \`Weight\` variables, we transform them to a real metric system :D

```{r}
nba<- nba %>% 
  mutate(
    Height_cm = Height * 2.54,
    Weight_kg = Weight * 0.453592,
    total_pts_carrer = PTS * G
    )

nba %>% 
  summarise(
    mean_pts = mean(PTS),
    max_pts = max(PTS),
    sd_pts = sd(PTS),
    avg_height = mean(Height_cm),
    avg_weight = mean(Weight_kg)
  )
```

For \`Position\` variable, we display its values and then filter the variable to include only those cases in which players are listed as Center, Forward-Center or Center-Forward.

```{r}
nba %>% 
  count(Position)

nba %>% 
  filter(Position %in% c("['Center', 'Forward']", "['Center']", "['Forward', 'Center']")) %>% 
  group_by(Position) %>% 
  summarise(AVG_PTS = mean(PTS))
```

Now we can explore our Centers

```{r}
nba %>% 
  filter(Position %in% c("['Center', 'Forward']", "['Center']", "['Forward', 'Center']"),
         Active == "True") %>%  
  group_by(Position) %>% 
  slice_max(order_by = PTS, n = 10) %>% 
  select(Position, Name, PTS, Active, Height_cm, Weight_kg) %>% 
  arrange(Position, desc(PTS))

```

Top 3 Centers

```{r}
nba %>% 
  filter(Position %in% c("['Center', 'Forward']", "['Center']", "['Forward', 'Center']"),
         Active == "True") %>%  
  group_by(Position) %>% 
  slice_max(order_by = PTS, n = 3) %>% 
  select(Position, Name, PTS, Active, Height_cm, Weight_kg, G,total_pts_carrer) %>% 
  arrange(Position, desc(PTS))
```

Before we jump into the inference analysis, we need to do some changes first:

-   Rename the variable \`G\` to \`games\`

-   Create a new variable called \`PTSCENTER\`, which contains the average points scored by active Centers in 2025

-   Change the reference category for Position variable to "\['Center'\]"

```{r}
nba<- nba %>% 
  mutate(
    games = G,
    PTSCENTER = case_when(
      Position %in% c("['Center', 'Forward']", "['Center']", "['Forward', 'Center']") & Active == "True" ~ PTS
    )
  )
```

We create a new dataset for active Centers

```{r}
nba_centers <- nba %>%
  filter(!is.na(PTSCENTER))
```

## Inference Analysis

For this part of the exercise, we estimate a linear regression model (OLS), which says as follows:

$$
AvgPointsScored_i = \beta_0 + \beta_1  Position_i + \beta_2Games_i + \beta_3Height_{cm_i} + \beta_4 Weight_{cm_i} + \\ \beta_5AvgAsist_i +  \beta_6AvgRebounds_i + \varepsilon_i
$$

We estimate model 1 and its fitted values

```{r}
m1<- lm(PTSCENTER ~ Position + games +Height_cm + Weight_kg + AST + TRB, data = nba_centers )
screenreg(m1)

nba_centers$fittedvalues <- predict(m1)
```

After that, we obtain the 10 highest fitted values and then we generate the plot

```{r}
nba_centers %>% 
  select(Name, PTS, fittedvalues, Position) %>% 
  slice_max(order_by = fittedvalues, n = 10) 


nba_centers %>%
  ggplot(aes(x = fittedvalues, y = PTS)) +
  geom_point(alpha = 0.6, color = "steelblue") +
  geom_abline(slope = 1, intercept = 0, linetype = "dashed", color = "red") +
  labs(
    title = "Comparación entre puntos reales y predichos",
    x = "Puntos predichos (fittedvalues)",
    y = "Puntos reales (PTS)"
  ) +
  theme_bw()
```

## Logit Model

For this part, we estimate a logit model. First things first, we obtain the average rebounds from the entire sample. Then we get the max and min rebounds values.

```{r}
nba_centers %>% 
  summarise(
    avg_rb = mean(TRB),
    max = max(TRB),
    min = min(TRB)
  )
```

In the following step, we create a new variable called \`rb_dummy\` in which returns 1 if the player have an average of 6 or greater rebounds per game, and 0 if the total rebounds are less than 6.

```{r}
nba_centers <- nba_centers %>% 
  mutate(
    rb_dummy = case_when(
      TRB >= 6 ~ 1,
      TRB < 6 ~ 0
    )
  )
```

We estimate the logit model

```{r}
m2<- glm(rb_dummy ~  PTSCENTER + Position + games +Height_cm + Weight_kg + AST, data = nba_centers,
         family = binomial (link = "logit") )
screenreg(m2)
```

INTERPRETAR

## Multilevel model (MEM)

Finally, we estimate a multilevel model. In this part, we expand the sample to include all positions and players in the data base whether actives or not. Lastly, for the level-2 variables, we select the top 5 schools with the highest players in order to simplify the exercise.

```{r}
nba<- nba %>% 
  mutate(
    CENTERS = Position %in% c("['Center', 'Forward']", "['Center']", "['Forward', 'Center']")
    )
  
top5_schools <- nba %>%
  count(School, sort = TRUE) %>%
  slice_max(n, n = 5) %>%
  pull(School)
  
 

nba_top5_schools <- nba %>% 
  filter(School %in% top5_schools) 

nba_top5_schools %>% 
  count(School, sort = T)
```

## Centrado

Our interest is to

```{r}
nba_top5_schools$games_cgm <- center(nba_top5_schools$games, type = "CGM")
nba_top5_schools$Height_cm_cgm <- center(nba_top5_schools$Height_cm, type = "CGM")
nba_top5_schools$Weight_kg_cgm <- center(nba_top5_schools$Weight_kg, type = "CGM")
nba_top5_schools$AST_cgm <- center(nba_top5_schools$AST, type = "CGM")
nba_top5_schools$TRB_cgm <- center(nba_top5_schools$TRB, type = "CGM")
```

We estimate three models. First, the

```{r}
mnulo<- lmer(PTS ~ 1 + (1|School), data = nba_top5_schools)

m3<- lmer(PTS ~ Position + games_cgm + Height_cm_cgm + Weight_kg_cgm + AST_cgm + TRB_cgm + (1|School)  , data = nba_top5_schools)


m4<- lmer(PTS ~ Position + games_cgm + Height_cm_cgm + Weight_kg_cgm + AST_cgm + TRB_cgm + (TRB_cgm|School)  , data = nba_top5_schools)
screenreg(list(mnulo,m3, m4), single.row = T)
```

Now we calculate the ICC

```{r}
ICC = 0.93/(0.93+29.32) * 100
print(ICC)

```

Anova

```{r}
anova(m3,m4)
```

qqmath graph

```{r}
qqmath(ranef(mnulo, condVar = TRUE))
```

We plot model 3

```{r}
ranefs_m3 <- ranef(m3)$School
ranefs_m3$School <- rownames(ranefs_m3)
colnames(ranefs_m3) <- c("intercept_ranef", "School")

fixefs_m3 <- fixef(m3)
intercept_fixed <- fixefs_m3["(Intercept)"]
slope_fixed <- fixefs_m3["TRB_cgm"]


TRB_seq <- seq(from = min(nba_top5_schools$TRB_cgm, na.rm = TRUE),
               to = max(nba_top5_schools$TRB_cgm, na.rm = TRUE),
               length.out = 100)

line_data_m3 <- expand.grid(
  TRB_cgm = TRB_seq,
  School = unique(nba_top5_schools$School)
)

line_data_m3 <- left_join(line_data_m3, ranefs_m3, by = "School")

line_data_m3 <- line_data_m3 %>%
  mutate(
    intercept_total = intercept_fixed + intercept_ranef,
    # Pendiente fija, igual para todas las escuelas
    slope_total = slope_fixed,
    PTS_pred = intercept_total + slope_total * TRB_cgm
  )

ggplot(nba_top5_schools, aes(x = TRB_cgm, y = PTS, color = School)) +
  geom_point(alpha = 0.6) +
  geom_line(data = line_data_m3, aes(x = TRB_cgm, y = PTS_pred, color = School), size = 1) +
  facet_wrap(~ School) +
  labs(
    title = "Fitted values m3 (random intercept, fixed slope)",
    x = "Centering TRB (Grand Mean)",
    y = "PTS",
    color = "School"
  ) +
  theme_bw()

```

Model 4 plot

```{r}
nba_top5_schools$PTS_pred <- predict(m4, re.form = NULL)
ranefs <- ranef(m4)$School
ranefs$School <- rownames(ranefs)
colnames(ranefs) <- c("intercept_ranef", "TRB_cgm_ranef", "School")


fixefs <- fixef(m4)
intercept_fixed <- fixefs["(Intercept)"]
slope_fixed <- fixefs["TRB_cgm"]


# Crear un grid de valores centrados de TRB
TRB_seq <- seq(from = min(nba_top5_schools$TRB_cgm, na.rm = TRUE),
               to = max(nba_top5_schools$TRB_cgm, na.rm = TRUE),
               length.out = 100)

# Expandir para cada escuela
line_data <- expand.grid(
  TRB_cgm = TRB_seq,
  School = unique(nba_top5_schools$School)
)

# Juntar con efectos aleatorios
line_data <- left_join(line_data, ranefs, by = "School")


# Calcular pendiente e intercepto total por escuela
line_data <- line_data %>%
  mutate(
    intercept_total = intercept_fixed + intercept_ranef,
    slope_total = slope_fixed + TRB_cgm_ranef,
    PTS_pred = intercept_total + slope_total * TRB_cgm
  )

ggplot(nba_top5_schools, aes(x = TRB_cgm, y = PTS, color = School)) +
  geom_point(alpha = 0.6) +
  geom_line(data = line_data, aes(x = TRB_cgm, y = PTS_pred, color = School), size = 1) +
  facet_wrap(~ School) +
  labs(
    title = "Fitted values m4 (random intercept, random slope)",
    x = "Centering TRB (Grand Mean)",
    y = "PTS",
    color = "School"
  ) +
  theme_bw()
```

Lastly, we obtain the intercepts and slopes of each model.

```{r}
# Para m3 (solo intercepto aleatorio)
ranefs_m3 <- ranef(m3)$School %>% 
  as.data.frame() %>% 
  rename(intercept_ranef = `(Intercept)`) %>% 
  mutate(School = rownames(ranef(m3)$School)) %>% 
  select(School, intercept_ranef)

fixefs_m3 <- fixef(m3)
intercept_fixed_m3 <- fixefs_m3["(Intercept)"]
slope_fixed_m3 <- fixefs_m3["TRB_cgm"]

ranefs_m3 <- ranefs_m3 %>%
  mutate(
    intercept_total = intercept_fixed_m3 + intercept_ranef,
    slope_fixed = slope_fixed_m3
  )

print("Interceptos y pendiente fija por escuela modelo m3:")
print(ranefs_m3)



```

```{r}
ranefs_m4 <- ranef(m4)$School %>% 
  as.data.frame() %>% 
  rename(
    intercept_ranef = `(Intercept)`,
    slope_ranef = TRB_cgm
  ) %>% 
  mutate(School = rownames(ranef(m4)$School)) %>% 
  select(School, intercept_ranef, slope_ranef)

fixefs_m4 <- fixef(m4)
intercept_fixed_m4 <- fixefs_m4["(Intercept)"]
slope_fixed_m4 <- fixefs_m4["TRB_cgm"]

ranefs_m4 <- ranefs_m4 %>%
  mutate(
    intercept_total = intercept_fixed_m4 + intercept_ranef,
    slope_total = slope_fixed_m4 + slope_ranef
  )

print("Interceptos y pendientes (random slopes) por escuela modelo m4:")
print(ranefs_m4)
```